In [3]:
from pycdoexpr import cdoexpr

# generate cdo map wind level expr (https://www.jodc.go.jp/data_format/weather-code.html)
cexpr = cdoexpr()
wind_level_bins = [ 0.3,  1.6,  3.4,  5.5,  8. , 10.8, 13.9, 17.2,\
    20.8, 24.5, 28.5, 32.6, 36.9, 41.4, 46.1, 50.9, 56. , 61.3]
wind_level = range(0,len(wind_level_bins)+1)
expr = cexpr.digitize('WIND', wind_level_bins, wind_level, right=False)
f"cdo -expr,'WIND_LEVEL={expr}' infile outfile"


"cdo -expr,'WIND_LEVEL=((WIND>=61.3))? 18:(((WIND>=56.0))? 17:(((WIND>=50.9))? 16:(((WIND>=46.1))? 15:(((WIND>=41.4))? 14:(((WIND>=36.9))? 13:(((WIND>=32.6))? 12:(((WIND>=28.5))? 11:(((WIND>=24.5))? 10:(((WIND>=20.8))? 9:(((WIND>=17.2))? 8:(((WIND>=13.9))? 7:(((WIND>=10.8))? 6:(((WIND>=8.0))? 5:(((WIND>=5.5))? 4:(((WIND>=3.4))? 3:(((WIND>=1.6))? 2:(((WIND>=0.3))? 1:0)))))))))))))))))' infile outfile"

In [4]:
s = '''
if PRE1H > 0.001:
    if TEM2 >= 3:
        if PRE1H < 0.1:
            WW = 51
        elif PRE1H < 2.5:
            WW = 61
        elif PRE1H < 8:
            WW = 62
        else:
            WW = 63
    elif TEM2 >=0:
        if PRE1H < 2.5:
            WW = 66
        else:
            WW = 67
    else:
        if PRE1H < 0.1:
            WW = 71
        elif PRE1H < 0.2:
            WW = 73
        else:
            WW = 75
else:
    if VIS > 10000:
        if TCC > 80:
            WW = 3
        elif TCC > 40:
            WW = 2
        else:
            WW = 0
    elif VIS >= 1000:
        if RHU2 > 80:
            WW = 45
        elif RHU2 > 50:
            WW = 48
        else:
            WW = 31
    else:
        if WS10 < 1:
            WW = 45
        else:
            if RHU2 >=50:
                WW = 45
            else:
                WW = 34
'''
expr = cexpr.conditions(s, verbose=True)
f"cdo expr,'WW={expr}' infile outfile"



                                                                        _________________________________________________________________________________________PRE1H > 0.001____________________________________________
                                                                       /                                                                                                                                                  \
               ___________________________________________________TEM2 >= 3_______________________________                                                                                 ___________________________VIS > 10000_________________________________________________
              /                                                                                           \                                                                               /                                                                                       \
     ___P

"cdo expr,'WW=((PRE1H > 0.001))? (((TEM2 >= 3))? (((PRE1H < 0.1))? ( 51): (((PRE1H < 2.5))? ( 61): (((PRE1H < 8))? ( 62): ( 63)))): (((TEM2 >=0))? (((PRE1H < 2.5))? ( 66): ( 67)): (((PRE1H < 0.1))? ( 71): (((PRE1H < 0.2))? ( 73): ( 75))))): (((VIS > 10000))? (((TCC > 80))? ( 3): (((TCC > 40))? ( 2): ( 0))): (((VIS >= 1000))? (((RHU2 > 80))? ( 45): (((RHU2 > 50))? ( 48): ( 31))): (((WS10 < 1))? ( 45): (((RHU2 >=50))? ( 45): ( 34)))))' infile outfile"